In [6882]:
#Using Pandas as the data processing package
import pandas as pd
import xlrd

#Using plotly as the visualization package
import plotly.express as px
import plotly.graph_objects as go

#Using Dash coponents for building the app
from dash import Dash, dcc, html, Input, Output, callback

#Using the dash bootstrap components for formatting
import dash_bootstrap_components as dbc;
import numpy as np


In [6883]:
# Import all the datasets into dataframes
StateGdp = pd.read_excel('4.GDP_NY_TX_OH.xlsx')
UsEmp = pd.read_excel("Employment_US.xlsx")
StateEmp = pd.read_excel("Employment_States.xlsx")
StateHousing = pd.read_excel("Housing_data.xlsx")
UsHousing = pd.read_excel("Mortgage_Delinquency.xlsx")
DiffSect = pd.read_excel("DifferentialSectors.xlsx")

# Correct the number into thousands in the datasets
StateEmp['TotalNonFarm'] = StateEmp['TotalNonFarm'].apply(lambda x:x*1000)

# Adjustments to the DiffSect dataframe.
# Calculate the annual job growth rate by state in differential sectors
DiffSect['Year'] = DiffSect['Date'].dt.year
AvgDiffSect = DiffSect.groupby(['Year','State']).agg(
    RetailEmp=('RetailEmployees', 'mean'),
    RAEmp=('RealEstateEmployees', 'mean'),
).reset_index()
AvgDiffSect['RetailGrowth'] = AvgDiffSect['RetailEmp'].pct_change(periods=1).mul(100)
AvgDiffSect['RealEstateGrowth'] = AvgDiffSect['RAEmp'].pct_change(periods=1).mul(100)
AvgDiffSect['Year']=pd.to_datetime(AvgDiffSect.Year, format='%Y')

# Adjustments to the UsEmp dataframe
# Calculate the annual averages for the variables in the dataframe.
UsEmp['Year'] = UsEmp['Date'].dt.year
UsEmp = UsEmp.groupby(['Year']).agg(
    JobOpeningRate=('JobOpeningRate', 'mean'),
    UnemploymentRate=('UnemploymentRate', 'mean'),
    PermanentJobLosers = ('PermanentJobLosses', 'mean')
).round(1).reset_index()
UsEmp['PermanentJobLosers'] = UsEmp['PermanentJobLosers'].apply(lambda x:x*1000)
UsEmp['Year']=pd.to_datetime(UsEmp.Year, format='%Y')

In [6884]:
# Define TimePeriod function for Recession, Normality and COVID depending on year
def TimePeriod(year):
    
    period = ""    
    
    if (year <= pd.Timestamp('2009-12-31')):  
        period = "Recession"
    elif(year >=  pd.Timestamp('2020-01-01')):
        period = "COVID"
    else:
        period = "Normality"

    return period
   
   
# Add the Period column to StateGdp, StateEmp and StateHousing dataframes based on the year of each row
StateGdp['Period'] = StateGdp["Date"].apply(lambda theYear: TimePeriod(theYear));
StateEmp['Period'] = StateEmp["Date"].apply(lambda theYear: TimePeriod(theYear));
StateHousing['Period'] = StateHousing["Date"].apply(lambda theYear: TimePeriod(theYear));

In [6885]:
# Store the three unique states
States = sorted(StateGdp.State.unique())
firstState = States[0];

# Store the unique periods 
Period = sorted(StateGdp.Period.unique());

In [6886]:
# Initialize the app and set a app title
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, dbc_css],  suppress_callback_exceptions=True)
app.title = "Impact of Recession and COVID"

In [6887]:
# Define variable Introtabcontent to store the intoduction tab content
Introtabcontent =  html.Div([
    html.P("The Great Recession in 2008 and the COVID-19 pandemic are two significnt events that caused severe economic downturns. " 
                        "This visual analysis aims to reveal the effects of these events on employment and the housing market at both the "
                        "national level and within the states of New York, Ohio, and Texas. Multiple indicators in both sectors are explored "
                        "against the timeline to understand the widespread repercussions of the economic strain experienced during these crises.", 
                        className="card-text")
],className="col-10")

In [6888]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise Job opening, Initial Claims, Total non farm employess Charts and Callback
JobOpStateChart = dcc.Graph()
ICalimsStateChart = dcc.Graph()
NonFarmEmpSatesChart = dcc.Graph()

#Callback for Job opening Chart
@callback(
        Output(JobOpStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateJobOpChart(selectedState):
    
    mask = StateEmp["State"] == selectedState; 
    Title = "Number of Job Openings in: " + selectedState;

    JobOpFig = px.area(
        StateEmp[mask],
        x="Date",
        y="JobOpenings",
        template="simple_white",
        title=Title,
        labels=dict(Date = "Year", JobOpenings = "Average Job Openings(Thousands)",)
    )
    JobOpFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    JobOpFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    JobOpFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    # JobOpFig.update_traces(mode='lines', fill='tozeroy', fillcolor='lightblue')
    return JobOpFig



#Callback for Initial Claims Chart
@callback(
        Output(ICalimsStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateICalimsChart(selectedState):
    
    mask = StateEmp["State"] == selectedState; 
    Title = "Initial Unemployment Claims in: " + selectedState;
         
    ICalimsFig = px.line(
        StateEmp[mask],
        x="Date",
        y="InitialClaims",
        template="simple_white",
        line_shape="spline",
        title=Title,
        markers=True,
        labels=dict(Date = "Year", InitialClaims = "Average Initial Claims (Thousands)",)
    )
    ICalimsFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    ICalimsFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    ICalimsFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray",
                    range=[0,110000])      
    )
    
    ICalimsFig.update_traces(line_color='steelblue', 
                            line_width=2,
                            marker = dict(color="darkblue", size = 8)
    )
    return ICalimsFig


#Callback for Total nonfarm employess Chart
@callback(
        Output(NonFarmEmpSatesChart, "figure"), 
        Input(dropdown, "value")
)

def updateNonFarmChart(selectedState):
    
    mask = StateEmp["State"] == selectedState; 
    Title = "Number of NonFarm Employees in: " + selectedState;

    NonFarmFig = px.line(
        StateEmp[mask],
        x="Date",
        y="TotalNonFarm",
        template="simple_white",
        line_shape="spline",
        markers=True,
        title=Title,
        labels=dict(Date = "Year", TotalNonFarm = "NonFarm Employees (Millions)",)
    )
    NonFarmFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    NonFarmFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    NonFarmFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    
    NonFarmFig.update_traces(line_color='steelblue', 
                            line_width=2,
                            marker = dict(color="darkblue", size = 8)
    )
    return NonFarmFig    

In [6889]:
# declare a variable called UnemploymentTabContent1 to hold the content (html, input and output components) of this tab in the page.
UnemploymentTabContent1 = html.Div([
    html.H2(
        "Annual Job Openings, NonFarm Employees & Initial Unemployment Claims", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Annual Job Openings", className="card-title"),
                html.P(
                    "The average annual Job Openings fell across the three states in both the Great Recession and COVID. "
                    "While the recovery during recession was slow, it was much faster after COVID.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            JobOpStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Initial Unemployment Claims", className="card-title"),
                html.P(
                    "The number of initial unemployment claims increased during both the recession and the COVID-19 pandemic. "
                    "The numbers were significantly higher during the COVID-19 period, mainly due to a shorter timeframe available for "
                    "addressing the economic challenges posed by the pandemic.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            ICalimsStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Number of NonFarm Employees", className="card-title"),
                html.P(
                    "The total number of nonfarm employees declined steadily across all three states during the recession. "
                    "However, the downturn was more pronounced during COVID, yet it swiftly rebounded to pre-pandemic levels.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            NonFarmEmpSatesChart
        ],className="col-8")
    ])
])

In [6890]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise Median Income, Continued Unemployment claims and Poverty rate charts and Callback
MedianIncomeChart = dcc.Graph()
CClaimsStateChart = dcc.Graph()
PovertyStateChart = dcc.Graph()

#Callback for State median income Chart
@callback(
        Output(MedianIncomeChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateMedianIncomeChart(selectedState):
    
    mask = StateEmp["State"] == selectedState;
    Title = "Median Income in: " + selectedState;
    
    MedianIncomeFig = px.line(
        StateEmp[mask],
        x="Date",
        y="MedianIncome",
        template="simple_white",
        title=Title,
        markers=True,
        line_shape="spline",
        labels=dict(Date = "Year", MedianIncome = "Median Income ($)",)
    )
    MedianIncomeFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    MedianIncomeFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    MedianIncomeFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
        
    )
    MedianIncomeFig.update_traces(line_color='steelblue', 
                                line_width=2,
                                marker = dict(color="darkblue", size = 10),
    )
    return MedianIncomeFig

#Callback for State continued claims Chart
@callback(
        Output(CClaimsStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateCClaimsChart(selectedState):
    
    mask = StateEmp["State"] == selectedState; 
    Title = "Continued Unemployment Claims in: " + selectedState;
         
    CClaimsFig = px.bar(
        StateEmp[mask],
        x="Date",
        y="ContinuedClaims",
        color="ContinuedClaims",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
       # markers=True,
        labels=dict(Date = "Year", InitialClaims = "Average Continued Claims (Millions)",)
    )
    CClaimsFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    CClaimsFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    CClaimsFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
        
    )
    return CClaimsFig

#Callback for State poverty rate Chart
@callback(
        Output(PovertyStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStatePovertyChart(selectedState):
    
    mask = StateEmp["State"] == selectedState;
    Title = "Poverty Rate in: " + selectedState;
         
    PovertyFig = px.bar(
        StateEmp[mask],
        x="Date",
        y="PovertyRate",
        color="PovertyRate",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
       # markers=True,
        labels=dict(Date = "Year", PovertyRate = "Poverty Rate (%)",)
    )
    PovertyFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    PovertyFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    PovertyFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
        
    )
    return PovertyFig

In [6891]:
# declare a variable called UnemploymentTabContent2 to hold the content (html, input and output components) of this tab in the page.
UnemploymentTabContent2 = html.Div([
    html.H2(
        "Continued Unemployment Claims, Poverty Rate and Median Income by State", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Continued Unemployment Claims", className="card-title"),
                html.P(
                    "The number of continued unemployment claims followed the trend of initial "
                    "claims during both the recession and the COVID-19 pandemic. " 
                    "These numbers were much larger as more people started experiencing job loss as the events progressed. " 
                    "All three states experienced the same but New York had the highest numbers.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            CClaimsStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Poverty Rate", className="card-title"),
                html.P(
                    "For all three states the poverty rate increased as the Great recession progressed. " 
                    "During COVID, Texas and Ohio saw a slight decline in poverty rate.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            PovertyStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Median Income", className="card-title"),
                html.P(
                    "The median income fell during both recession and COVID in the three states. "
                    "The recovery after recession took many years.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            MedianIncomeChart
        ],className="col-8")
    ])
])

In [6892]:
# US Job opening rate and unemployment rate chart
UsEmpFig = px.bar(
    UsEmp,
    x="Year",
    y=["JobOpeningRate", "UnemploymentRate"],
    template="simple_white",
    title="Job Opening Rate, Unemployment Rate - US"     
)

UsEmpFig.add_vrect(
    x0="2007-12-01", x1="2009-07-30",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
)

UsEmpFig.add_vrect(
    x0="2020-01-01", x1="2020-12-31",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
)

UsEmpFig.update_layout(
    xaxis_title="Year",
    yaxis_title="Rate in %",
    legend_title="Measure Name",
    barmode= "group",
    bargap=0.40,
    legend = dict(font=dict(size=10, color="gray")),
    xaxis = dict(nticks=10,
    tickformat="%Y",
    linecolor="lightgray"
    ),
    
    yaxis =dict(linecolor="lightgray"),
)
UsEmpFig.update_traces(hovertemplate=None)
#UsEmpFig.update_traces(mode="lines+markers", hovertemplate=None)
UsEmpFig.update_layout(hovermode="x unified")


# US permanent job losers chart
UsJobLossFig = px.bar(
    UsEmp,
    x="Year",
    y="PermanentJobLosers",
    color="PermanentJobLosers",
    template="simple_white",
    color_continuous_scale=["lightblue", "darkblue"],
    title="Permanent Job Losers (16 yaers and over) - US",
    labels=dict(Date = "Year", PermanentJobLosers = "Permanent Job Losers (Millions)",),
)
UsJobLossFig.add_vrect(
    x0="2007-12-01", x1="2009-07-30",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
)

UsJobLossFig.add_vrect(
    x0="2020-01-01", x1="2020-12-31",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
)

UsJobLossFig.update_layout(
    bargap=0.40,
    legend = dict(font=dict(size=8, color="gray")),
    xaxis = dict(nticks=10,
    tickformat="%Y",
    linecolor="lightgray"
    ),
    
    yaxis =dict(linecolor="lightgray")
)

In [6893]:
# declare a variable called UnemploymentTabContent4 to hold the content (html, input and output components) of this tab in the page.
UnemploymentTabContent4 = html.Div([
    html.H2(
        "Unemployment Rate, Job Opening Rate and Number of Permanent Job Losers in the USA", 
        className="mt-4 px-4 align-center"), 
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Unemployment Rate and Job Opening Rate", className="card-title"),
                html.P(
                    "The unemployment rate and the job opening rate follow an inverse trend, as expected. " 
                    "On both of them, the impact of the recession was sharper, and the recovery was much slower compared to COVID.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            dcc.Graph(figure=UsEmpFig)
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Permanent Job Losers", className="card-title"),
                html.P(
                    "The number of permanent job losers (16 years and over) was much higher during the recession and took years to decline.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            dcc.Graph(figure=UsJobLossFig)
        ],className="col-8")
    ]),
])

In [6894]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise Home value, home owenership rate and new building permits Charts and Callback
HomeValueStateChart = dcc.Graph()
HomeOwnRateStateChart = dcc.Graph()
NewPermitsStateChart = dcc.Graph()

# Callback for State home value chart
@callback(
        Output(HomeValueStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateHomeValueChart(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "Average Home Value Index (Zillow) in: " + selectedState;
    
    HomeValueFig = px.line(
        StateHousing[mask],
        x="Date",
        y="AvgHomeValue",
        template="simple_white",
        title=Title,
        markers=True,
        line_shape="spline",
        labels=dict(Date = "Year", AvgHomeValue = "Average Home Value ($)",)
    )
    HomeValueFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeValueFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeValueFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")    
    )
    
    HomeValueFig.update_traces(line_color='steelblue', 
                                line_width=2,
                                marker = dict(color="darkblue", size = 10),
    )
    
    HomeValueFig.update_traces(mode="lines+markers", hovertemplate=None)
    HomeValueFig.update_layout(hovermode="x unified")
    return HomeValueFig

# Callback for state homeownership rate
@callback(
        Output(HomeOwnRateStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateHomeOwnRateChart(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "Homeownership Rate in: " + selectedState;
         
    HomeOwnFig = px.bar(
        StateHousing[mask],
        x="Date",
        y="HomeownershipRate",
        color="HomeownershipRate",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
        labels=dict(Date = "Year", HomeownershipRate = "Homeownership Rate (%)",)
    )
    
    HomeOwnFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeOwnFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeOwnFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
        
    )
    
    return HomeOwnFig


# Callback for new building permits chart
@callback(
        Output(NewPermitsStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateNewPermitsChart(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "New Building Permits issued in: " + selectedState;
         
    PermitsFig = px.bar(
        StateHousing[mask],
        x="Date",
        y="PermitsHousing",
        color="PermitsHousing",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
        labels=dict(Date = "Year", PermitsHousing = "Average Units (Thousands)",)
    )
    PermitsFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    PermitsFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    PermitsFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )

    return PermitsFig

In [6895]:
# declare a variable called HousingTabContent1 to hold the content (html, input and output components) of this tab in the page.
HousingTabContent1 = html.Div([
    html.H2(
        "Home Value Index, Homeownership Rate and New Building Permits Issued", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Home Value Index (Zillow)", className="card-title"),
                html.P(
                    "The burst of the housing bubble during the recession caused a sharp drop in home " 
                    "values across the states. However, COVID saw a steady increase in home value "
                    "as housing inventory fell and mortgage rates came down to historical lows.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            HomeValueStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Homeownership Rate", className="card-title"),
                html.P(
                    "The struggles in keeping homes led to a decline in homeownership rate during the recession. "
                    "However, COVID restrictions and low mortgage rates increased homeownership rates ",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            HomeOwnRateStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Building Permits", className="card-title"),
                html.P(
                    "The housing crisis during the recession led to fewer number of new buidling permits. "
                    "During COVID, the permits issued saw an uptick",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            NewPermitsStateChart
        ],className="col-8")
    ])
])

In [6896]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise Rental vacancy rate and home vacancy rate Charts and Callback
RentVacRateStateChart = dcc.Graph()
HomeVacRateStateChart = dcc.Graph()

# Callback for rental vacancy rate chart
@callback(
        Output(RentVacRateStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateRentVacRateChart(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "Rental Vacancy Rate in: " + selectedState;
         
    RentVacFig = px.bar(
        StateHousing[mask],
        x="Date",
        y="RentalVacancyRate",
        color="RentalVacancyRate",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
        labels=dict(Date = "Year", RentalVacancyRate = "Rental Vacancy Rate (%)",)
    )
    RentVacFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    RentVacFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    RentVacFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    
    return RentVacFig

# Callback for home vacancy rate chart
@callback(
        Output(HomeVacRateStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateHomeVacRateChart(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "Home Vacancy Rate in: " + selectedState;
         
    HomeVacFig = px.bar(
        StateHousing[mask],
        x="Date",
        y="HomeVacancyRate",
        color="HomeVacancyRate",
        template="simple_white",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
        labels=dict(Date = "Year", HomeVacancyRate = "Home Vacancy Rate (%)",)
    )
    HomeVacFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeVacFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeVacFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    return HomeVacFig

In [6897]:
# declare a variable called HousingTabContent2 to hold the content (html, input and output components) of this tab in the page.
HousingTabContent2 = html.Div([
    html.H2(
        "Rental Vacancy Rate and Home Vacancy Rate", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Rental Vacancy Rate", className="card-title"),
                html.P(
                    "The rental vacancy rates grew during recession. During covid, " 
                    "it decreased steadily in all three states because of shortage of rentals.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            RentVacRateStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Home Vacancy Rate", className="card-title"),
                html.P(
                    "The home vacancy rates grew during recession. During covid, " 
                    "it decreased steadily in all three states because of lower inventory and increased demand of homes.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            HomeVacRateStateChart
        ],className="col-8")
    ]),
])

In [6898]:
#US 30 year fixed mortgage rate chart
UsMortgageFig = px.line(
    UsHousing,
    x="Date",
    y="MortgageRate",
    template="simple_white",
    markers=True,
    line_shape="spline",
    title="30 Year Fixed Mortgage Rate - US",
    labels=dict(Date = "Year", MortgageRate = "Mortgage Rate (%)",)  
)

UsMortgageFig.add_vrect(
    x0="2007-12-01", x1="2009-07-30",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
)

UsMortgageFig.add_vrect(
    x0="2020-01-01", x1="2020-12-31",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
)

UsMortgageFig.update_layout(
    legend = dict(font=dict(size=8, color="gray")),
    xaxis = dict(nticks=10,
    tickformat="%Y",
    linecolor="lightgray"
    ),
    
    yaxis =dict(linecolor="lightgray")
)

UsMortgageFig.update_traces(line_color='steelblue', 
                            line_width=2,
                            marker = dict(color="darkblue", size = 8)
)


# US Delinquency rate chart
UsDelinquencyRateFig = px.bar(
    UsHousing,
    x="Date",
    y="DelinquencyRate",
    color="DelinquencyRate",
    template="simple_white",
    color_continuous_scale=["lightblue", "darkblue"],
    title="Delinquency Rate - US",
    labels=dict(Date = "Year", DelinquencyRate = "Delinquency Rate (%)",),
)

UsDelinquencyRateFig.add_vrect(
    x0="2007-12-01", x1="2009-07-30",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
)

UsDelinquencyRateFig.add_vrect(
    x0="2020-01-01", x1="2020-12-31",
    fillcolor="linen", opacity=0.7,
    layer="below", line_width=0,
    label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
)

UsDelinquencyRateFig.update_layout(
    bargap=0.40,
    legend = dict(font=dict(size=8, color="gray")),
    xaxis = dict(nticks=10,
    tickformat="%Y",
    linecolor="lightgray"
    ),
    
    yaxis =dict(linecolor="lightgray")
)

In [6899]:
# declare a variable called HousingTabContent3 to hold the content (html, input and output components) of this tab in the page.
HousingTabContent3 = html.Div([
    html.H2(
        "30 Year Fixed Mortgage Rate and Delinquency Rate in the USA", 
        className="mt-4 px-4 align-center"), 
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("30 Year Fixed Mortgage Rate", className="card-title"),
                html.P(
                    "Mortgage rates spiked during the recession. " 
                    "It took years for the rates to come down to the lowest of the period in 2012. "
                    "Covid saw the lowest mortgage rate in December 2020. This led to a increased " 
                    "demand for homes. The rates started to spike soon after the pandemic.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            dcc.Graph(figure=UsMortgageFig)
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Delinquency Rate", className="card-title"),
                html.P(
                    "Delinquency rates during recession rose significantly due to loan defaults and foreclosures. "
                    "During Covid the rates declined steadily due to low mortgage rates and a renewed " 
                    "interest in home ownership. ",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            dcc.Graph(figure=UsDelinquencyRateFig)
        ],className="col-8")
    ]),
])

In [6900]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise GDP, unemployment rate and homeownership rate Charts and Callback
GdpStateChart = dcc.Graph()
UnempRateStateChart = dcc.Graph()
HomeOwnRateStateChart1 = dcc.Graph()

#Callback for state GDP chart
@callback(
        Output(GdpStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateGdpChart(selectedState):
    
    mask = StateGdp["State"] == selectedState;
    Title = "GDP in: " + selectedState;
    
    GdpStateFig = px.area(
        StateGdp[mask],
        x="Date",
        y="GDP",
        template="simple_white",
        title=Title,
        labels=dict(Date = "Year", GDP = "GDP($)",)
    )
    GdpStateFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    GdpStateFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    GdpStateFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    
    return GdpStateFig


# Callback for state unemployment rate chart
@callback(
        Output(UnempRateStateChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateURateChart(selectedState):
    
    mask = StateEmp["State"] == selectedState;
    Title = "Unemployment Rate in: " + selectedState;
    
    URateStateFig = px.bar(
        StateEmp[mask],
        x="Date",
        y="UnemploymentRate",
        template="simple_white",
        color= "UnemploymentRate",
        color_continuous_scale=["lightblue", "darkblue"],
        title=Title,
        labels=dict(Date = "Year", UnemploymentRate = "Unemployment Rate (%)",)
    )
    
    URateStateFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    URateStateFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    URateStateFig.update_layout(
        bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),

        yaxis =dict(linecolor="lightgray")
    )
    return URateStateFig


# Callback for state homeownership rate
@callback(
        Output(HomeOwnRateStateChart1, "figure"), 
        Input(dropdown, "value")
)

def updateStateHomeOwnRateChart1(selectedState):
    
    mask = StateHousing["State"] == selectedState;
    Title = "Homeownership Rate in: " + selectedState;
         
    HomeOwnFig = px.line(
        StateHousing[mask],
        x="Date",
        y="HomeownershipRate",
        line_shape="spline",
        template="simple_white",
        markers=True,
        title=Title,
        labels=dict(Date = "Year", HomeownershipRate = "Homeownership Rate (%)",)
    )
    
    HomeOwnFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeOwnFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.5,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    HomeOwnFig.update_layout(
       # bargap=0.40,
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
        
    )
    
    HomeOwnFig.update_traces(line_color='steelblue', 
                            line_width=2,
                            marker = dict(color="darkblue", size = 8)
    )   
    
    return HomeOwnFig

In [6901]:
# declare a variable called LeadLagTabContent to hold the content (html, input and output components) of this tab in the page.
LeadLagTabContent = html.Div([
    html.H2(
        "GDP and Unemployment Rate", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("GDP", className="card-title"),
                html.P(
                    "The GDP decreased slightly accross the three states during the recession and COVID. " 
                    "GDP was a leading indicator for both recession and COVID.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            GdpStateChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Unemployment Rate", className="card-title"),
                html.P(
                    "Unemployment rate rose sharply leading to recession making it a leading indicator. On the other hand, " 
                    "during COVID, it saw a sharp increase due to mass job losses which made it a lagging indicator.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            UnempRateStateChart
        ],className="col-8")
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Homeownership Rate", className="card-title"),
                html.P(
                    "The homeownership rate showed steady decline leading upto the recession. It was a leading indicator "
                    "for the Great recession as the housing crisis was unfolding. But for COVID, the decline started after " 
                    "the pandemic due to shortage of inventory and higher demand, making it a lagging indicator. ",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            HomeOwnRateStateChart1
        ],className="col-8")
    ])
])

In [6902]:
# Define the dropdown component for states
dropdown = dcc.Dropdown(
    
    options=States,
    value=firstState,
    clearable=False
)

#State wise Retail job growth rate and real estate job growth rate Charts and Callback
RetailGrowthChart = dcc.Graph()
RealEstateGrowthChart = dcc.Graph()


# Callback for state Retail job growth rate chart
@callback(
        Output(RetailGrowthChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateRetailGrowthChart(selectedState):
    
    mask = AvgDiffSect["State"] == selectedState;
    Title = "Retail Job Growth Rate in: " + selectedState;
    
    RetailGrowthFig = px.line(
        AvgDiffSect[mask],
        x="Year",
        y="RetailGrowth",
        template="simple_white",
        title=Title,
        markers=True,
        line_shape="spline",
        labels=dict(Year = "Year", RetailGrowth = "Retail Job Growth Rate (%)",)
    )
    RetailGrowthFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )

    RetailGrowthFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    RetailGrowthFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    
    RetailGrowthFig.update_traces(marker = dict(color="darkblue", symbol="diamond", size = 12),
    )
    
    return RetailGrowthFig


# Callback for real estate job growth rate Chart
@callback(
        Output(RealEstateGrowthChart, "figure"), 
        Input(dropdown, "value")
)

def updateStateRealEstateGrowthChart(selectedState):
    
    mask = AvgDiffSect["State"] == selectedState;
    Title = "Real Estate Job Growth Rate in: " + selectedState;
    
    RealEstateGrowthFig = px.line(
        AvgDiffSect[mask],
        x="Year",
        y="RealEstateGrowth",
        template="simple_white",
        title=Title,
        markers=True,
        line_shape="spline",
        labels=dict(Year = "Year", RealEstateGrowth = "Real Estate Job Growth Rate (%)",)
    )
    RealEstateGrowthFig.add_vrect(
        x0="2007-12-01", x1="2009-07-30",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="Recession", textposition="top center", font=dict(size=8, color="gray")),
    )
    
    RealEstateGrowthFig.add_vrect(
        x0="2020-01-01", x1="2020-12-31",
        fillcolor="linen", opacity=0.7,
        layer="below", line_width=0,
        label=dict(text="COVID", textposition="top center", font=dict(size=8, color="gray")),
    )

    RealEstateGrowthFig.update_layout(
        legend = dict(font=dict(size=8, color="gray")),
        xaxis = dict(nticks=10,
        tickformat="%Y",
        linecolor="lightgray"
        ),
        
        yaxis =dict(linecolor="lightgray")
    )
    
    RealEstateGrowthFig.update_traces(marker = dict(color="darkblue", symbol="diamond", size = 12),
    )
    
    return RealEstateGrowthFig

In [6903]:
# declare a variable called DiffSectTabContent to hold the content (html, input and output components) of this tab in the page.
DiffSectTabContent = html.Div([
    html.H2(
        "Differentiaal Sectors in States", 
        className="mt-4 px-4 align-center"), 

    dbc.Row([
        dbc.Col([
            html.Div([
                dbc.Label("Select a State", className="pl-4"),
                dropdown,
            ], className="col-3")
        ], className="my-4"),
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Retail Growth Rate", className="card-title"),
                html.P(
                    "The retail job growth rate experienced a decline during both the Great Recession and COVID. " 
                    "Texas was the least impacted state, showing a minor slowdown during these events, " 
                    "after which it continued its upward trend.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
    
        dbc.Col([
            RetailGrowthChart
        ],className="col-8")
    ]),
        
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Real Estate Growth Rate", className="card-title"),
                html.P(
                    "The Real Estate job growth rate experienced a decline during both the Great Recession and COVID. " 
                    "Texas was the least impacted state, showing a minor slowdown during these events, " 
                    "after which it continued its upward trend.",
                    className="card-text",
                )
            ], className="mt-4")
        ],className="col-4"),
          
        dbc.Col([
            RealEstateGrowthChart
        ],className="col-8")
    ]),
])

In [6904]:
# Define the Header for the dashboard
header = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(
                        [
                            html.Div(
                                [
                                    html.H3("Comparing the impact of the Great Recession and the COVID-19 Pandemic", style={"color": "#fff"}),
                                    html.P("Employment and Housing Sector", style={"color": "#fff"}),
                                ],
                                id="app-title",
                            ),
                        ],
                        md="auto",
                    )
                ],
                align="center",
            ),
        ],
        fluid=True,
    ),
    # dark=True,
    color="dark",
    sticky="top",
)

# The card header consists of the list of tabs with individual label and tab id
# The card body contains a callback to id card-content
card = dbc.Card(
    [
        dbc.CardHeader(
            dbc.Tabs(
                [
                    dbc.Tab(
                        label ="Introduction", 
                        tab_id="tab-1",
                    )  ,
                    dbc.Tab(
                        label="Employment: Beginning of Events",
                        tab_id="tab-2",
                    ),
                    dbc.Tab(
                        label="Employment: Continued Impacts",
                        tab_id="tab-3",
                    ),
                    dbc.Tab(
                        label="Employment: US",
                        tab_id="tab-4",
                    ),
                    dbc.Tab(
                        label="Housing: Multi Dimensions",
                        tab_id="tab-5",
                    ),
                    dbc.Tab(
                        label="Housing: Vacancy Rates",
                        tab_id="tab-6",
                    ),
                    dbc.Tab(
                        label="Housing: US ",
                        tab_id="tab-7",
                    ),
                    dbc.Tab(
                        label="Differential Impacts",
                        tab_id="tab-8",
                    ),
                    dbc.Tab(
                        label="Leaders & Laggards",
                        tab_id="tab-9",
                    )
                ],
                id="card-tabs",
                active_tab="tab-1",
            )
        ),
        dbc.CardBody(id="card-content"),
    ]
)

# Callback to the tabs in the card body.
@app.callback(
    Output("card-content", "children"), [Input("card-tabs", "active_tab")]
)

def tab_content(active_tab):
    if active_tab == 'tab-1':
        return Introtabcontent
    elif active_tab == 'tab-2':
        return UnemploymentTabContent1
    elif active_tab == 'tab-3':
        return UnemploymentTabContent2
    elif active_tab == 'tab-4':
        return UnemploymentTabContent4
    elif active_tab == 'tab-5':
        return HousingTabContent1
    elif active_tab == 'tab-6':
        return HousingTabContent2
    elif active_tab == 'tab-7':
        return HousingTabContent3
    elif active_tab == 'tab-8':
        return DiffSectTabContent
    elif active_tab == 'tab-9':
        return LeadLagTabContent

# App Layout
app.layout = html.Div([
    header,
    dbc.Container(
        [
            html.Div(card, className="my-4"),
        
        ],
        fluid=True,
    )
]);

In [6905]:
# Run the app at http://127.0.0.1:8052/ when running locally
if __name__ == "__main__":
    app.run_server(
        debug=True,
        port=8052
    )